<a href="https://colab.research.google.com/github/milanpavikevik/GenreClassifier/blob/FINAL/data_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# basic modules we need for scraping
import warnings                        # To ignore any warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%pylab inline
import os
import glob
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Audio
import librosa
import librosa.display

# Scraping with BS4 module
from PIL import Image
from google.colab import files
from bs4 import BeautifulSoup
import requests
import urllib
import urllib.request

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# install the api for downloading YT files with Python
!pip install youtube_dl

     |████████████████████████████████| 1.8MB 5.1MB/s 


In [ ]:
# import the modules used for downloading
import youtube_dl
import time
import random

In [ ]:
#agent for successful web scraping
usr_agent = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive',
}

In [ ]:
# install, import and set the Selenium module
# YT is a very responsive WebSite => BS4 is not compatible
# We need to use Selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [ ]:
# connect to the YT WebSite
from selenium.webdriver.common.keys import Keys
wd.get("https://www.youtube.com/")

In [ ]:
# Structures for easier data handling
genres = ['pop', 'rock','hip-hop/rap','electronic', 'techno','makedonski narodni pesni','macedonian folk','metal','jazz','classical','opera']
dict_urls = {'pop':[],'rock':[], 'hip-hop/rap':[], 'electronic':[], 'techno':[], 'makedonski narodni pesni':[],'macedonian folk':[],'metal':[],'jazz':[],'classical':[], 'opera':[]}
print(dict_urls)
dict_url = {'pop':None,'rock':None, 'hip-hop/rap':None, 'electronic':None, 'techno':None, 'makedonski narodni pesni':None,'macedonian folk':None,'metal':None,'jazz':None,'classical':None, 'opera':None}
print(dict_url)

{'pop': [], 'rock': [], 'hip-hop/rap': [], 'electronic': [], 'techno': [], 'makedonski narodni pesni': [], 'macedonian folk': [], 'metal': [], 'jazz': [], 'classical': [], 'opera': []}
{'pop': None, 'rock': None, 'hip-hop/rap': None, 'electronic': None, 'techno': None, 'makedonski narodni pesni': None, 'turbo folk': None, 'metal': None, 'jazz': None, 'classical': None, 'opera': None}


In [ ]:
# obtain links for playlists for all genres
for genre in genres:
  wd.get("https://www.youtube.com/")
  try:
    #try:
    # find the search bar and type the genre
    search_box = wd.find_element_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div/div[1]/input').send_keys(genre)
    wd.find_element_by_xpath('//*[@id="search-icon-legacy"]').click() # click the search button
    time.sleep(2)
    wd.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer').click() # click the button for options
    time.sleep(1)
    # choose that we are ONLY searching for playlists
    wd.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[3]/a').click()
    wd.get(wd.current_url)
    wd.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-playlist-renderer[1]/ytd-playlist-thumbnail/a').click()
    print(wd.current_url)
    # add the URL for the link on the respective Key in the dictionary
    dict_url[genre] = wd.current_url
  except:
    # if something goes wrong, just notify which genre it is
    print(f'There is a problem with the genre {genre}')
  

https://www.youtube.com/watch?v=SlPhMPnQ58k&list=PL4o29bINVT4EG_y-k5jGoOu3-Am8Nvi10
https://www.youtube.com/watch?v=kXYiU_JCYtU&list=PL6Lt9p1lIRZ311J9ZHuzkR5A3xesae2pk
https://www.youtube.com/watch?v=meFxq3-mNEc&list=PLxhnpe8pN3TlMilD9JLcwNmjqf2J47cRU
https://www.youtube.com/watch?v=foE1mO2yM04&list=PL7wr9BYcCCyNb0IhqqebdLEMkklMSOttA
https://www.youtube.com/watch?v=Lgs9QUtWc3M&list=PLnuL2ltPXh02ThmKDdTrxCH3EjOupmRq2
https://www.youtube.com/watch?v=jAHKy_rExWM&list=PL9629A9FA6085C586
https://www.youtube.com/watch?v=qeAEdU4p5Cg&list=PL2ILAkvYYMWz16csJSeOvy59qsMtQGtoQ
https://www.youtube.com/watch?v=xnKhsTXoKCI&list=PLhQCJTkrHOwSX8LUnIMgaTq3chP1tiTut
https://www.youtube.com/watch?v=RPfFhfSuUZ4&list=PL8F6B0753B2CCA128
https://www.youtube.com/watch?v=9E6b3swbnWg&list=PLz9q8tuDHcRfQ--1tUrZj4xC1pShtz9cD
https://www.youtube.com/watch?v=Uau48wh5CeI&list=PLDBt7LIJrBCVjfCZnQxBqnMtQ74IJW0uT


In [ ]:
# print the genres with their playlist's URL
for key in dict_url:
  print(key,dict_url[key])

pop https://www.youtube.com/watch?v=SlPhMPnQ58k&list=PL4o29bINVT4EG_y-k5jGoOu3-Am8Nvi10
rock https://www.youtube.com/watch?v=kXYiU_JCYtU&list=PL6Lt9p1lIRZ311J9ZHuzkR5A3xesae2pk
hip-hop/rap https://www.youtube.com/watch?v=meFxq3-mNEc&list=PLxhnpe8pN3TlMilD9JLcwNmjqf2J47cRU
electronic https://www.youtube.com/watch?v=foE1mO2yM04&list=PL7wr9BYcCCyNb0IhqqebdLEMkklMSOttA
techno https://www.youtube.com/watch?v=Lgs9QUtWc3M&list=PLnuL2ltPXh02ThmKDdTrxCH3EjOupmRq2
makedonski narodni pesni https://www.youtube.com/watch?v=jAHKy_rExWM&list=PL9629A9FA6085C586
turbo folk https://www.youtube.com/watch?v=qeAEdU4p5Cg&list=PL2ILAkvYYMWz16csJSeOvy59qsMtQGtoQ
metal https://www.youtube.com/watch?v=xnKhsTXoKCI&list=PLhQCJTkrHOwSX8LUnIMgaTq3chP1tiTut
jazz https://www.youtube.com/watch?v=RPfFhfSuUZ4&list=PL8F6B0753B2CCA128
classical https://www.youtube.com/watch?v=9E6b3swbnWg&list=PLz9q8tuDHcRfQ--1tUrZj4xC1pShtz9cD
opera https://www.youtube.com/watch?v=Uau48wh5CeI&list=PLDBt7LIJrBCVjfCZnQxBqnMtQ74IJW0uT


In [ ]:
# iterate through the playlists and obtain the links for all the songs separately
cntr=0
categoriSoLinkovi=[]
for key in dict_url:
  genre = key
  searchurl = dict_url[key]
  response = requests.get(searchurl, headers=usr_agent)
  # list of song youtube urls 
  listofIdSongs = []
  html = response.text
  # parse the website
  soup = BeautifulSoup(html, 'html.parser')
  #find the links from the source page
  results = soup.findAll('script')
  fullText = str(results[-5])
  # default youtube url without id
  youtubeUrl = "https://www.youtube.com/watch?v="
  posleden=""
  #print(fullText)
  for i in range (0,len(fullText)):
    if (fullText[i]=='"' and fullText[i+1]=='v' and fullText[i+2]=='i' and fullText[i+3]=='d' and fullText[i+4]=='e' and fullText[i+5]=='o' and fullText[i+6]=='I' and fullText[i+7]=='d' and fullText[i+8]=='"' ):
      if posleden != fullText[(i+11):(i+22)]:
        listofIdSongs.append (youtubeUrl + fullText[(i+11):(i+22)])
      posleden= fullText[(i+11):(i+22)]
  cntr+=len(listofIdSongs)
  dict_urls[genre] = listofIdSongs
  print (f'sample links from {len(listofIdSongs)} {genre} songs from the playlist:',listofIdSongs[0:120])  
  categoriSoLinkovi.append(listofIdSongs)
print(f'There are in total {cntr} samples!')  


sample links from 244 pop songs from the playlist: ['https://www.youtube.com/watch?v=oygrmJFKYZY', 'https://www.youtube.com/watch?v=HCjNJDNzw8Y', 'https://www.youtube.com/watch?v=SlPhMPnQ58k', 'https://www.youtube.com/watch?v=aJOTlE1K90k', 'https://www.youtube.com/watch?v=SlPhMPnQ58k', 'https://www.youtube.com/watch?v=aJOTlE1K90k', 'https://www.youtube.com/watch?v=JPJjwHAIny4', 'https://www.youtube.com/watch?v=vZA5heWazIQ', 'https://www.youtube.com/watch?v=nIjVuRTm-dc', 'https://www.youtube.com/watch?v=09R8_2nJtjg', 'https://www.youtube.com/watch?v=rtOvBOTyX00', 'https://www.youtube.com/watch?v=Iic_OOZKexw', 'https://www.youtube.com/watch?v=hT_nvWreIhg', 'https://www.youtube.com/watch?v=cBVGlBWQzuc', 'https://www.youtube.com/watch?v=ANS9sSJA9Yc', 'https://www.youtube.com/watch?v=i8VKeDheoUM', 'https://www.youtube.com/watch?v=2Vv-BfVoq4g', 'https://www.youtube.com/watch?v=bo_efYhYU2A', 'https://www.youtube.com/watch?v=5Wiio4KoGe8', 'https://www.youtube.com/watch?v=lp-EO5I60KA', 'https:/

In [ ]:
"""
=== These are only 3 of the observed classes ===
=== DO NOT download all the songs in 1 GO ===
=== You may face request errors ===
"""
classical=['https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=9E6b3swbnWg',
           'https://www.youtube.com/watch?v=EYydbECKQJw', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EFJ7kDva7JE',
           'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ', 'https://www.youtube.com/watch?v=y1dbbrfekAM',
           'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI', 'https://www.youtube.com/watch?v=K0k1WCPWZ8Q',
           'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 'https://www.youtube.com/watch?v=4Tr0otuiQuU',
           'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=mIYzp5rcTvU', 'https://www.youtube.com/watch?v=9uKVI6_Fm4g',
           'https://www.youtube.com/watch?v=VB6SIKl8Md0', 'https://www.youtube.com/watch?v=FS6o3qFimsc', 'https://www.youtube.com/watch?v=mGQLXRTl3Z0',
           'https://www.youtube.com/watch?v=GRxofEmo3HA', 'https://www.youtube.com/watch?v=_ioc6sdgugo', 'https://www.youtube.com/watch?v=dN1XUV6QclU',
           'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=CvFH_6DNRCY',
           'https://www.youtube.com/watch?v=_4IRMYuE1hI', 'https://www.youtube.com/watch?v=YyknBTm_YyM', 'https://www.youtube.com/watch?v=GXFSK0ogeg4',
           'https://www.youtube.com/watch?v=VbxgYlcNxE8', 'https://www.youtube.com/watch?v=z7rxl5KsPjs', 'https://www.youtube.com/watch?v=DUmq1cpcglQ',
           'https://www.youtube.com/watch?v=7lC1lRz5Z_s', 'https://www.youtube.com/watch?v=GGU1P6lBW6Q', 'https://www.youtube.com/watch?v=iCEDfZgDPS8',
           'https://www.youtube.com/watch?v=5poSw7tFLB4', 'https://www.youtube.com/watch?v=TZCfydWF48c', 'https://www.youtube.com/watch?v=fBA-38mzabs',
           'https://www.youtube.com/watch?v=36Y_ztEW1NE', 'https://www.youtube.com/watch?v=vCHREyE5GzQ', 'https://www.youtube.com/watch?v=F5zg_af9b8c',
           'https://www.youtube.com/watch?v=mgHxmAsINDk', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=M_VCbnqbwwA',
           'https://www.youtube.com/watch?v=_jBLyIQvNf0', 'https://www.youtube.com/watch?v=Sy-yugPw_X8', 'https://www.youtube.com/watch?v=Nx5c_JZIM6M',
           'https://www.youtube.com/watch?v=hZY5DBmgC_A', 'https://www.youtube.com/watch?v=7cb1QmTkOAI', 'https://www.youtube.com/watch?v=7nVmFlSV1ok',
           'https://www.youtube.com/watch?v=WeAc0oh818s', 'https://www.youtube.com/watch?v=6_UPjZyxn0Q', 'https://www.youtube.com/watch?v=WZ4wOpFstT8',
           'https://www.youtube.com/watch?v=hopaQjQFUYw', 'https://www.youtube.com/watch?v=PhcYvl0Cxh0', 'https://www.youtube.com/watch?v=D3qbOTgEJOk',
           'https://www.youtube.com/watch?v=3m5ohobcKb8', 'https://www.youtube.com/watch?v=BLSXaJIaKfo', 'https://www.youtube.com/watch?v=QSomvC6rwgU',
           'https://www.youtube.com/watch?v=bMipey2Q4hE', 'https://www.youtube.com/watch?v=2AsGdP5jeKA', 'https://www.youtube.com/watch?v=hmNAvgQ9ZjY', 
           'https://www.youtube.com/watch?v=x_CLwjp2Zu8', 'https://www.youtube.com/watch?v=7jKvoXJ5KOY', 'https://www.youtube.com/watch?v=gxyA5bgpels', 
           'https://www.youtube.com/watch?v=JlORKi6A_SE', 'https://www.youtube.com/watch?v=LXzA4oquapY', 'https://www.youtube.com/watch?v=DbGVdKSKoEQ', 
           'https://www.youtube.com/watch?v=2shqZIeVBHU', 'https://www.youtube.com/watch?v=pTbwQ6G-bP0', 'https://www.youtube.com/watch?v=1ZzjpnSdeVA',
           'https://www.youtube.com/watch?v=w4gaYNVCywg', 'https://www.youtube.com/watch?v=cbzYaf-_DbI', 'https://www.youtube.com/watch?v=5XPYkSzxYhY',
           'https://www.youtube.com/watch?v=ZS-HWIFyLsE', 'https://www.youtube.com/watch?v=VrOosUb0shw', 'https://www.youtube.com/watch?v=FK-KC2aQpcI',
           'https://www.youtube.com/watch?v=9BDlqlhcCIk', 'https://www.youtube.com/watch?v=pUBymqCBs4g', 'https://www.youtube.com/watch?v=9E6b3swbnWg',
           'https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=cbzYaf-_DbI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo',
           'https://www.youtube.com/watch?v=36Y_ztEW1NE', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EYydbECKQJw',
           'https://www.youtube.com/watch?v=EFJ7kDva7JE', 'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ',
           'https://www.youtube.com/watch?v=y1dbbrfekAM', 'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI',
           'https://www.youtube.com/watch?v=K0k1WCPWZ8Q', 'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 
           'https://www.youtube.com/watch?v=4Tr0otuiQuU', 'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=9E6b3swbnWg',
           'https://www.youtube.com/watch?v=azao0lM0CTE', 'https://www.youtube.com/watch?v=mxK1lLMEzX0']

opera=['https://www.youtube.com/watch?v=sMVFXfoaS3w', 'https://www.youtube.com/watch?v=CJiiBq8UnIY', 'https://www.youtube.com/watch?v=Uau48wh5CeI',
      'https://www.youtube.com/watch?v=kpUaqE00y70', 'https://www.youtube.com/watch?v=cWc7vYjgnTs', 'https://www.youtube.com/watch?v=Uau48wh5CeI',
      'https://www.youtube.com/watch?v=sMVFXfoaS3w', 'https://www.youtube.com/watch?v=g4LOpDVkzD0', 'https://www.youtube.com/watch?v=6S5xHMbU-mA',
      'https://www.youtube.com/watch?v=2J7JM0tGgRY', 'https://www.youtube.com/watch?v=w77SFM7Fksc', 'https://www.youtube.com/watch?v=5ZDGPKcEPJ8',
      'https://www.youtube.com/watch?v=a5lYyMNxR4Q', 'https://www.youtube.com/watch?v=qt_OkgSOrkU', 'https://www.youtube.com/watch?v=xN-JCdM4or0',
      'https://www.youtube.com/watch?v=mqIljeeoLXE', 'https://www.youtube.com/watch?v=mhhkGyJ092E', 'https://www.youtube.com/watch?v=Bg-An7TQBbI',
      'https://www.youtube.com/watch?v=Tos3o_GWED4', 'https://www.youtube.com/watch?v=phBThlPTBEg', 'https://www.youtube.com/watch?v=6R5-FXVHruQ',
      'https://www.youtube.com/watch?v=QJ2ZM-3YTg0', 'https://www.youtube.com/watch?v=mONTSRzdRZw', 'https://www.youtube.com/watch?v=JJwecS3kNCM',
      'https://www.youtube.com/watch?v=pWffUAg8zyc', 'https://www.youtube.com/watch?v=Uau48wh5CeI', 'https://www.youtube.com/watch?v=sMVFXfoaS3w',
      'https://www.youtube.com/watch?v=2oaIcChc4tI', 'https://www.youtube.com/watch?v=rvQvcLXHWI4', 'https://www.youtube.com/watch?v=3omEZ3XyM74',
      'https://www.youtube.com/watch?v=JhLxK2xGqqo', 'https://www.youtube.com/watch?v=nFs1fDOmHVw', 'https://www.youtube.com/watch?v=PxZJzuVG6W8',
      'https://www.youtube.com/watch?v=y5N4l2MaccI', 'https://www.youtube.com/watch?v=0LgkkwyUK_A', 'https://www.youtube.com/watch?v=8rjMS13CgT4',
      'https://www.youtube.com/watch?v=CJiiBq8UnIY', 'https://www.youtube.com/watch?v=Bmi-_oT2kF8', 'https://www.youtube.com/watch?v=Iv0W04G44Ls',
      'https://www.youtube.com/watch?v=01f8_3LdJdo', 'https://www.youtube.com/watch?v=SbkBlg7a78U', 'https://www.youtube.com/watch?v=lfgTHyZ4S5o',
      'https://www.youtube.com/watch?v=d3xhWNVIV3g', 'https://www.youtube.com/watch?v=mgSH4A-V9HQ', 'https://www.youtube.com/watch?v=-rV3FvlvXa8',
      'https://www.youtube.com/watch?v=LqcD2c5WFIc', 'https://www.youtube.com/watch?v=0Wlcr2bIKmk', 'https://www.youtube.com/watch?v=rQoATtAODzY', 
      'https://www.youtube.com/watch?v=Hyc_BrXA_Ow', 'https://www.youtube.com/watch?v=-KY2TlUHggw', 'https://www.youtube.com/watch?v=Z3Zq8KXUAo0', 
      'https://www.youtube.com/watch?v=N7RLPGr832o', 'https://www.youtube.com/watch?v=540gS4gru7w', 'https://www.youtube.com/watch?v=3J99JXVYfwQ', 
      'https://www.youtube.com/watch?v=Uau48wh5CeI', 'https://www.youtube.com/watch?v=sMVFXfoaS3w', 'https://www.youtube.com/watch?v=3J99JXVYfwQ', 
      'https://www.youtube.com/watch?v=CJiiBq8UnIY', 'https://www.youtube.com/watch?v=JhLxK2xGqqo', 'https://www.youtube.com/watch?v=Uau48wh5CeI', 
      'https://www.youtube.com/watch?v=cWc7vYjgnTs', 'https://www.youtube.com/watch?v=sMVFXfoaS3w', 'https://www.youtube.com/watch?v=g4LOpDVkzD0', 
      'https://www.youtube.com/watch?v=6S5xHMbU-mA', 'https://www.youtube.com/watch?v=2J7JM0tGgRY', 'https://www.youtube.com/watch?v=w77SFM7Fksc', 
      'https://www.youtube.com/watch?v=5ZDGPKcEPJ8', 'https://www.youtube.com/watch?v=a5lYyMNxR4Q', 'https://www.youtube.com/watch?v=qt_OkgSOrkU', 
      'https://www.youtube.com/watch?v=xN-JCdM4or0', 'https://www.youtube.com/watch?v=mqIljeeoLXE', 'https://www.youtube.com/watch?v=mhhkGyJ092E', 
      'https://www.youtube.com/watch?v=Uau48wh5CeI', 'https://www.youtube.com/watch?v=v_z-6NukGIw']

hip_hop_rap=['https://www.youtube.com/watch?v=JFm7YDVlqnI', 'https://www.youtube.com/watch?v=-V2PxXsctV0', 'https://www.youtube.com/watch?v=meFxq3-mNEc',
             'https://www.youtube.com/watch?v=x8mAyKX9muE', 'https://www.youtube.com/watch?v=meFxq3-mNEc', 'https://www.youtube.com/watch?v=ohD6cyB8RI4',
             'https://www.youtube.com/watch?v=YsOweFtVlvU', 'https://www.youtube.com/watch?v=Zx_DBvChpvM', 'https://www.youtube.com/watch?v=l0U7SxXHkPY',
             'https://www.youtube.com/watch?v=GErG9femMQk', 'https://www.youtube.com/watch?v=pwBFOuCrdr4', 'https://www.youtube.com/watch?v=qJlMkMtK7-8',
             'https://www.youtube.com/watch?v=n5LWsc-qCdA', 'https://www.youtube.com/watch?v=iXUPeEuM1CM', 'https://www.youtube.com/watch?v=HIwAI05Y1fU', 
             'https://www.youtube.com/watch?v=HVbOKVnXooI', 'https://www.youtube.com/watch?v=78yLkf2J6JI', 'https://www.youtube.com/watch?v=T8pi91qWnRw', 
             'https://www.youtube.com/watch?v=oBYf6gpVvRA', 'https://www.youtube.com/watch?v=QryoOF5jEbc', 'https://www.youtube.com/watch?v=GA370tAjzv8', 
             'https://www.youtube.com/watch?v=-tmSSWl-ZoA', 'https://www.youtube.com/watch?v=3CxtK7-XtE0', 'https://www.youtube.com/watch?v=X9x7JKGa27I',
             'https://www.youtube.com/watch?v=meFxq3-mNEc', 'https://www.youtube.com/watch?v=JFm7YDVlqnI', 'https://www.youtube.com/watch?v=GErG9femMQk', 
             'https://www.youtube.com/watch?v=HnUYD1U1qTw', 'https://www.youtube.com/watch?v=0C80BSgjb8M', 'https://www.youtube.com/watch?v=83xBPCw5hh4', 
             'https://www.youtube.com/watch?v=IdT-hcjdVbg', 'https://www.youtube.com/watch?v=JStxkxgSlx8', 'https://www.youtube.com/watch?v=DmInwHOt8cY', 
             'https://www.youtube.com/watch?v=0opZqh_TprM', 'https://www.youtube.com/watch?v=oXxvjwynJvs', 'https://www.youtube.com/watch?v=HIwAI05Y1fU', 
             'https://www.youtube.com/watch?v=BsytXAJ6Wr8', 'https://www.youtube.com/watch?v=s4Mn4rc0YPg', 'https://www.youtube.com/watch?v=jV1v2Cy1w30', 
             'https://www.youtube.com/watch?v=H2Po5uiZMco', 'https://www.youtube.com/watch?v=JxAlZOn4QhM', 'https://www.youtube.com/watch?v=dYk-yCsuTUo', 
             'https://www.youtube.com/watch?v=wdrmK0DtztU', 'https://www.youtube.com/watch?v=GrAchTdepsU', 'https://www.youtube.com/watch?v=fyIcQ1Xl-rs', 
             'https://www.youtube.com/watch?v=G6XthQpk8uk', 'https://www.youtube.com/watch?v=u0LaoQks5mY', 'https://www.youtube.com/watch?v=YNyFAro1E5M', 
             'https://www.youtube.com/watch?v=CiCT7NCP6BY', 'https://www.youtube.com/watch?v=DnMg6HykXzg', 'https://www.youtube.com/watch?v=jUIrolORx6M', 
             'https://www.youtube.com/watch?v=eTaC_pPpGL8', 'https://www.youtube.com/watch?v=3jI_-8UQmN0', 'https://www.youtube.com/watch?v=pPw_izFr5PA', 
             'https://www.youtube.com/watch?v=gZIsMh3IS7E', 'https://www.youtube.com/watch?v=tIYdsBjN9-k', 'https://www.youtube.com/watch?v=7KTt4_M09Rw', 
             'https://www.youtube.com/watch?v=BUJ_hyOMIq8', 'https://www.youtube.com/watch?v=rJZynxvJnlI', 'https://www.youtube.com/watch?v=I6FmwBPDT-w', 
             'https://www.youtube.com/watch?v=C5i-UnuUKUI', 'https://www.youtube.com/watch?v=VjMKmJDT_qU', 'https://www.youtube.com/watch?v=Qgduhk26sIw', 
             'https://www.youtube.com/watch?v=6MCSr65d9Xc', 'https://www.youtube.com/watch?v=Q9pjm4cNsfc', 'https://www.youtube.com/watch?v=jWQNqlZ47Ec', 
             'https://www.youtube.com/watch?v=_VDGysJGNoI', 'https://www.youtube.com/watch?v=deMLg4FPFhQ', 'https://www.youtube.com/watch?v=-iNWEwLfkv8',
             'https://www.youtube.com/watch?v=1KlNDZpCkHc', 'https://www.youtube.com/watch?v=AY1bA23hGMU', 'https://www.youtube.com/watch?v=knjPA5BtEII',
             'https://www.youtube.com/watch?v=Cel1TAel-eY', 'https://www.youtube.com/watch?v=OQ6R4TU5Vwo', 'https://www.youtube.com/watch?v=LPnnX7hOcQM', 
             'https://www.youtube.com/watch?v=0Bl_IYEHWaU', 'https://www.youtube.com/watch?v=VxLHOwp2XSo', 'https://www.youtube.com/watch?v=ted2pAnAnQc', 
             'https://www.youtube.com/watch?v=m8jZunevga8', 'https://www.youtube.com/watch?v=T8pi91qWnRw', 'https://www.youtube.com/watch?v=OZa3HyVLimQ', 
             'https://www.youtube.com/watch?v=Tk9l62Cl5EU', 'https://www.youtube.com/watch?v=l3Cj7Esqr0c', 'https://www.youtube.com/watch?v=MEUTlymw1eU', 
             'https://www.youtube.com/watch?v=aITIKxjFaL0', 'https://www.youtube.com/watch?v=n79N2Vk2Y48', 'https://www.youtube.com/watch?v=bE7AmqT0E80', 
             'https://www.youtube.com/watch?v=ddoFu4sDXpU', 'https://www.youtube.com/watch?v=V_jHc_n0p9c', 'https://www.youtube.com/watch?v=if1BAn17y3k', 
             'https://www.youtube.com/watch?v=iXUPeEuM1CM', 'https://www.youtube.com/watch?v=TjuWsp0Nnk0', 'https://www.youtube.com/watch?v=ZdhVDWcrh6c', 
             'https://www.youtube.com/watch?v=K5pJe2gse6I', 'https://www.youtube.com/watch?v=Qt6UcHJAeFo', 'https://www.youtube.com/watch?v=tN2j075NLQc', 
             'https://www.youtube.com/watch?v=Q5jnuOCJvK0', 'https://www.youtube.com/watch?v=oVEVrYq9v7M', 'https://www.youtube.com/watch?v=86p9_ycUyq8', 
             'https://www.youtube.com/watch?v=qBn_aZPcqJk', 'https://www.youtube.com/watch?v=eUe2cdhsL68', 'https://www.youtube.com/watch?v=o_smxk-P5p4', 
             'https://www.youtube.com/watch?v=5tOytaaALGQ', 'https://www.youtube.com/watch?v=rpubTIE3mdY', 'https://www.youtube.com/watch?v=xWggTb45brM', 
             'https://www.youtube.com/watch?v=Uz3zBZQzO5Q', 'https://www.youtube.com/watch?v=_3LuZx06QhQ', 'https://www.youtube.com/watch?v=ZD6C498MB4U', 
             'https://www.youtube.com/watch?v=R16itGXKp9k', 'https://www.youtube.com/watch?v=c2lUhNmdXkE', 'https://www.youtube.com/watch?v=8oaW16lGNxE', 
             'https://www.youtube.com/watch?v=aFkxqGZPlSA', 'https://www.youtube.com/watch?v=-uOdzsteXvk', 'https://www.youtube.com/watch?v=XRNjRcKZc1A', 
             'https://www.youtube.com/watch?v=hJPS6JTXAFM', 'https://www.youtube.com/watch?v=l2eGsg-Ab_8', 'https://www.youtube.com/watch?v=XnSD4J6XKLA', 
             'https://www.youtube.com/watch?v=nMDOSLEVdBw', 'https://www.youtube.com/watch?v=UNZqm3dxd2w', 'https://www.youtube.com/watch?v=r_0JjYUe5jo']




In [ ]:
# download the songs for the 3 genres
genre_list = [[classical,'classical'],[opera,'opera'],[hip_hop_rap,'hip_hop_rap']]
for genre in genre_list:
  for i in range (0,len(genre[0])):
      try:
          time.sleep(random.randint(10, 15)) # add this line to avoid request exceptions
          video_url=genre[0][i]
          video_info = youtube_dl.YoutubeDL().extract_info(
                  url=video_url, download=False
              )
          filename = f"/content/drive/My Drive/Songs/{genre[1]}/{video_info['title']}.mp3"
          options = {
              'format': 'bestaudio/best',
              'keepvideo': False,
              'outtmpl': filename,
              'postprocessors': [{
                  'key': 'FFmpegExtractAudio',
                  'preferredcodec': 'mp3',
                  'preferredquality': '192',
              }]
          }
          with youtube_dl.YoutubeDL(options) as ydl:
              ydl.download([video_info['webpage_url']])
      except:
          # if something goes wrong, notify us for which song exactly
          print(f'Song number {i} from genre:{genre} - could not download!')

[youtube] Zi8vJ_lMxQI: Downloading webpage
[youtube] Zi8vJ_lMxQI: Downloading webpage
[download] Destination: /content/drive/My Drive/Songs/classical/Mozart - Requiem.mp3
[download] 100% of 8.16MiB in 00:00
[ffmpeg] Post-process file /content/drive/My Drive/Songs/classical/Mozart - Requiem.mp3 exists, skipping
[youtube] wXQCPAR0EHo: Downloading webpage
[youtube] wXQCPAR0EHo: Downloading webpage
[download] Destination: /content/drive/My Drive/Songs/classical/Rachmaninov - Prelude in C Sharp Minor.mp3
[download] 100% of 4.88MiB in 00:00
[ffmpeg] Post-process file /content/drive/My Drive/Songs/classical/Rachmaninov - Prelude in C Sharp Minor.mp3 exists, skipping
[youtube] 9E6b3swbnWg: Downloading webpage
[youtube] 9E6b3swbnWg: Downloading webpage
[download] Destination: /content/drive/My Drive/Songs/classical/Chopin - Nocturne op.9 No.2.mp3
[download] 100% of 4.91MiB in 00:00
[ffmpeg] Post-process file /content/drive/My Drive/Songs/classical/Chopin - Nocturne op.9 No.2.mp3 exists, skippin

ERROR: This video is not available.


Song number 55 from genre:[['https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EYydbECKQJw', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EFJ7kDva7JE', 'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ', 'https://www.youtube.com/watch?v=y1dbbrfekAM', 'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI', 'https://www.youtube.com/watch?v=K0k1WCPWZ8Q', 'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 'https://www.youtube.com/watch?v=4Tr0otuiQuU', 'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=mIYzp5rcTvU', 'https://www.youtube.com/watch?v=9uKVI6_Fm4g', 'https://www.youtube.com/watch?v=VB6SIKl8Md0', 'https://www.youtube.com/watch?v=FS6o3qFimsc', 'https://www.youtube.com/watch?v

ERROR: This video is not available.


Song number 57 from genre:[['https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EYydbECKQJw', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EFJ7kDva7JE', 'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ', 'https://www.youtube.com/watch?v=y1dbbrfekAM', 'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI', 'https://www.youtube.com/watch?v=K0k1WCPWZ8Q', 'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 'https://www.youtube.com/watch?v=4Tr0otuiQuU', 'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=mIYzp5rcTvU', 'https://www.youtube.com/watch?v=9uKVI6_Fm4g', 'https://www.youtube.com/watch?v=VB6SIKl8Md0', 'https://www.youtube.com/watch?v=FS6o3qFimsc', 'https://www.youtube.com/watch?v

...

Song number 78 from genre:[['https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EYydbECKQJw', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EFJ7kDva7JE', 'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ', 'https://www.youtube.com/watch?v=y1dbbrfekAM', 'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI', 'https://www.youtube.com/watch?v=K0k1WCPWZ8Q', 'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 'https://www.youtube.com/watch?v=4Tr0otuiQuU', 'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=mIYzp5rcTvU', 'https://www.youtube.com/watch?v=9uKVI6_Fm4g', 'https://www.youtube.com/watch?v=VB6SIKl8Md0', 'https://www.youtube.com/watch?v=FS6o3qFimsc', 'https://www.youtube.com/watch?v

ERROR: pUBymqCBs4g: YouTube said: Unable to extract video data


Song number 79 from genre:[['https://www.youtube.com/watch?v=Zi8vJ_lMxQI', 'https://www.youtube.com/watch?v=wXQCPAR0EHo', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EYydbECKQJw', 'https://www.youtube.com/watch?v=9E6b3swbnWg', 'https://www.youtube.com/watch?v=EFJ7kDva7JE', 'https://www.youtube.com/watch?v=uELhJmYq2i0', 'https://www.youtube.com/watch?v=3mi145S-HfQ', 'https://www.youtube.com/watch?v=y1dbbrfekAM', 'https://www.youtube.com/watch?v=I0kVNMHo6fQ', 'https://www.youtube.com/watch?v=9_C6CTs0WhI', 'https://www.youtube.com/watch?v=K0k1WCPWZ8Q', 'https://www.youtube.com/watch?v=jgpJVI3tDbY', 'https://www.youtube.com/watch?v=O2deXwf4drE', 'https://www.youtube.com/watch?v=4Tr0otuiQuU', 'https://www.youtube.com/watch?v=-7jjo8UICjQ', 'https://www.youtube.com/watch?v=mIYzp5rcTvU', 'https://www.youtube.com/watch?v=9uKVI6_Fm4g', 'https://www.youtube.com/watch?v=VB6SIKl8Md0', 'https://www.youtube.com/watch?v=FS6o3qFimsc', 'https://www.youtube.com/watch?v

ERROR: Unable to extract JS player URL; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


Song number 108 from genre:[['https://www.youtube.com/watch?v=JFm7YDVlqnI', 'https://www.youtube.com/watch?v=-V2PxXsctV0', 'https://www.youtube.com/watch?v=meFxq3-mNEc', 'https://www.youtube.com/watch?v=x8mAyKX9muE', 'https://www.youtube.com/watch?v=meFxq3-mNEc', 'https://www.youtube.com/watch?v=ohD6cyB8RI4', 'https://www.youtube.com/watch?v=YsOweFtVlvU', 'https://www.youtube.com/watch?v=Zx_DBvChpvM', 'https://www.youtube.com/watch?v=l0U7SxXHkPY', 'https://www.youtube.com/watch?v=GErG9femMQk', 'https://www.youtube.com/watch?v=pwBFOuCrdr4', 'https://www.youtube.com/watch?v=qJlMkMtK7-8', 'https://www.youtube.com/watch?v=n5LWsc-qCdA', 'https://www.youtube.com/watch?v=iXUPeEuM1CM', 'https://www.youtube.com/watch?v=HIwAI05Y1fU', 'https://www.youtube.com/watch?v=HVbOKVnXooI', 'https://www.youtube.com/watch?v=78yLkf2J6JI', 'https://www.youtube.com/watch?v=T8pi91qWnRw', 'https://www.youtube.com/watch?v=oBYf6gpVvRA', 'https://www.youtube.com/watch?v=QryoOF5jEbc', 'https://www.youtube.com/watch?